In [ ]:
%load_ext rpy2.ipython

In [3]:
%%R
lapply(c("dplyr","Seurat","HGNChelper","openxlsx"), library, character.only = T)

load('../data/reference.RData')

sample_meta<-'AD_HS_00001	Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease	20	H.Sapiens	AD	Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex	Braak:0;0;0;2;2;2;2;6;6;6;0;0;0;2;2;2;2;6;6;6	ExcitatoryNeuron;InhibitoryNeuron;Oligodendrocyte;OPC;Astrocyte;Microglia;Endothelia;Pericyte	GSE147528	Single-nucleus RNA-seq	10x Genomics	Male	60;50;71;72;87;91;77;82;72;82;60;50;71;72;87;91;77;82;72;82	SRR11422700;SRR11422701;SRR11422702;SRR11422703;SRR11422704;SRR11422705;SRR11422706;SRR11422707;SRR11422708;SRR11422709;SRR11422710;SRR11422711;SRR11422712;SRR11422713;SRR11422714;SRR11422715;SRR11422716;SRR11422717;SRR11422718;SRR11422719'
split_meta<-unlist(strsplit(sample_meta, '\t'))
samples<-unlist(strsplit(split_meta[14],';'))
disease<-rep(split_meta[5],length(samples))
stage<-unlist(strsplit(unlist(strsplit(split_meta[7],':'))[2],';'))
gender<-rep(split_meta[12],length(samples))
age<-unlist(strsplit(split_meta[13],';'))

sample_meta<-data.frame(samples=samples,disease=disease,stage=stage,gender=gender,age=age)
sample_meta[sample_meta$stage=='0','disease']<-'Control'
rownames(sample_meta)<-sample_meta$samples

In [ ]:
%%R
library(stringr)
# args <- commandArgs(trailingOnly=TRUE)
args<-c('../data/3/scte','../data/3/cell_umap.txt','SRR11422700','/home/wdeng3/workspace/software/sc-type','../../universal_data/rmsk/rmsk_GRCh38.txt')
data_path<-args[1]
out_path<-args[2]
sample_tag<-args[3]
sctype_path<-args[4]
rmsk_path<-args[5]
rmsk<-read.table(rmsk_path,sep='\t',header=F)
to_remove<-unlist(rmsk[!(rmsk$V12 %in% c('LINE','SINE','LTR')),'V11'])
to_remove<-str_replace_all(to_remove,'-','\\.')

reps<-unlist(rmsk[rmsk$V12 %in% c('LINE','SINE','LTR'),'V11'])
reps<-str_replace_all(reps,'-','\\.')

create_seurat<-function(sample_tag){
    ## load data
    scte.data <- t(read.csv(paste0(data_path,'/',sample_tag,'/',sample_tag,'.csv'),check.names=F, row.names = 1))
    # remove repeats from gene expression matrix

    scte.gene<-scte.data[!(row.names(scte.data) %in% to_remove),]
    # normalize and scale data
    scte <- CreateSeuratObject(counts = scte.gene, project = sample_tag, min.cells = 3, min.features = 200)
    # normalize data
    scte[["percent.mt"]] <- PercentageFeatureSet(scte, pattern = "^MT.|^MT-")
    scte <- subset(scte, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
    scte <- SCTransform(scte, vars.to.regress = "percent.mt", verbose = FALSE)

    cell_meta<-scte@meta.data
    cell_meta$disease<-sample_meta[cell_meta$orig.ident,'disease']

    cell_meta$stage<-sample_meta[cell_meta$orig.ident,'stage']
    cell_meta$gender<-sample_meta[cell_meta$orig.ident,'gender']
    cell_meta$age<-sample_meta[cell_meta$orig.ident,'age']
    scte@meta.data<-cell_meta
    scte
}

scte<-create_seurat(sample_tag)

scte <- FindVariableFeatures(scte, selection.method = "vst", nfeatures = 2000)
# scale and run PCA
scte <- ScaleData(scte, features = rownames(scte))
scte <- RunPCA(scte, features = VariableFeatures(object = scte))

sdv<-Stdev(scte,reduction='pca')
sdv<-sdv[sdv>2]
npca<-length(sdv)

scte <- FindNeighbors(scte, dims = 1:npca)
scte <- FindClusters(scte, resolution = 0.8)
scte <- RunUMAP(scte, dims = 1:npca)


scte.anchors <- FindTransferAnchors(reference = reference, query = scte,
    dims = 1:30, reference.reduction = "pca")
scte.query <- MapQuery(anchorset = scte.anchors, reference = reference, query = scte,
    refdata = list(celltype = "broad.cell.type"), reference.reduction = "pca", reduction.model = "umap")
cell_meta<-scte.query@meta.data

cell_meta$UMAP_1<-scte.query@reductions[['umap']]@cell.embeddings[,1]
cell_meta$UMAP_2<-scte.query@reductions[['umap']]@cell.embeddings[,2]
scte.query@meta.data<-cell_meta
write.table(cell_meta,out_path,sep='\t',quote=F,row.names=T)

## get gene expression from scte_query
scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)

In [102]:
%%R
## get gene expression from scte_query
# scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
# scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
# scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)

In [202]:
import sys
import math
import pandas as pd
import numpy as np

sys.argv=['th','../data/3/cell_exp.txt','../www/mysql','../../universal_data/rmsk/rmsk_GRCh38.txt','../data/3/cell_umap.txt']
out_path=sys.argv[2]
cell_exp=pd.read_csv(sys.argv[1],sep='\t',index_col=0)
rmsk=pd.read_csv(sys.argv[3],sep='\t')
rmsk=rmsk.loc[rmsk['repClass'].isin(['LINE','SINE','LTR']),:]
cell_anno=pd.read_csv(sys.argv[4],sep='\t',index_col=0)

families=rmsk['repFamily'].unique()
subfams=rmsk['repName'].unique()

subfam_cell_count={x:list(set(cell_exp.loc[cell_exp[x]>0,:].index)) if x in cell_exp.columns else [] for x in subfams}
fam_dic={}
te2family={}
for x in families:
    fam_dic[x]=rmsk.loc[rmsk['repFamily']==x,'repName'].unique().tolist()
    te2family.update({y:x for y in fam_dic[x]})
    
fam_cell_count={x:[] for x in fam_dic}

for x in fam_dic:
    for y in fam_dic[x]:
        if y in subfam_cell_count:
            fam_cell_count[x].extend(subfam_cell_count[y])
for x in fam_dic:
    fam_cell_count[x]=list(set(fam_cell_count[x]))

subfam_cell_count_mtx=pd.DataFrame(np.zeros([len(subfams),len(['Ex','In','Oli','Opc','Ast','Mic','Per'])]),columns=['Ex','In','Oli','Opc','Ast','Mic','Per'],index=subfams)
for x in subfam_cell_count:
    for y in subfam_cell_count[x]:
        subfam_cell_count_mtx.loc[x,cell_anno.loc[y,'predicted.celltype']]+=1
fam_cell_count_mtx=pd.DataFrame(np.zeros([len(families),len(['Ex','In','Oli','Opc','Ast','Mic','Per'])]),columns=['Ex','In','Oli','Opc','Ast','Mic','Per'],index=families)
for x in fam_cell_count:
    for y in fam_cell_count[x]:
        fam_cell_count_mtx.loc[x,cell_anno.loc[y,'predicted.celltype']]+=1
    
subfam_mtx_out=open(out_path+'/subfam_cellcount.sql','w')
subfam_mtx_out.write('''CREATE DATABASE IF NOT EXISTS scARE;
USE scARE;
DROP TABLE IF EXISTS SUBFAM_CELL_COUNT;
CREATE TABLE SUBFAM_CELL_COUNT (
    ID INT NOT NULL AUTO_INCREMENT,
    TE varchar(255) NOT NULL,
    TE_FAM varchar(255) NOT NULL,
    Ex FLOAT NOT NULL,
    `In` FLOAT NOT NULL,
    Oli FLOAT NOT NULL,
    Opc FLOAT NOT NULL,
    Ast FLOAT NOT NULL,
    Mic FLOAT NOT NULL,
    Per FLOAT NOT NULL,
    PRIMARY KEY (ID)
);
''')
for i in range(subfam_cell_count_mtx.shape[0]):
    values=','.join([str(i) for i in subfam_cell_count_mtx.iloc[i,:]])
    te_fam=te2family[subfam_cell_count_mtx.index[i]]
    values='"'+subfam_cell_count_mtx.index[i]+'",'+f'"{te_fam}",'+values
    subfam_mtx_out.write(f'INSERT INTO SUBFAM_CELL_COUNT (TE,TE_FAM,Ex,`In`,Oli,Opc,Ast,Mic,Per) values({values});\n')
subfam_mtx_out.close()

fam_mtx_out=open(out_path+'/fam_cellcount.sql','w')
fam_mtx_out.write('''CREATE DATABASE IF NOT EXISTS scARE;
USE scARE;
DROP TABLE IF EXISTS FAM_CELL_COUNT;
CREATE TABLE FAM_CELL_COUNT (
    ID INT NOT NULL AUTO_INCREMENT,
    TE varchar(255) NOT NULL,
    Ex FLOAT NOT NULL,
    `In` FLOAT NOT NULL,
    Oli FLOAT NOT NULL,
    Opc FLOAT NOT NULL,
    Ast FLOAT NOT NULL,
    Mic FLOAT NOT NULL,
    Per FLOAT NOT NULL,
    PRIMARY KEY (ID)
);
''')
for i in range(fam_cell_count_mtx.shape[0]):
    values=','.join([str(i) for i in fam_cell_count_mtx.iloc[i,:]])
    values='"'+fam_cell_count_mtx.index[i]+'",'+values
    fam_mtx_out.write(f'INSERT INTO FAM_CELL_COUNT (TE,Ex,`In`,Oli,Opc,Ast,Mic,Per) values({values});\n')
fam_mtx_out.close()



In [ ]:
# ## assign cell type according to ScType tuturial

## load sc type and database
# source(paste0(sctype_path,"/R/gene_sets_prepare.R"))
# source(paste0(sctype_path,"/R/sctype_score_.R"))
# gs_list = gene_sets_prepare(paste0(sctype_path,'/ScTypeDB_full.xlsx'), "Brain" )
# # get cell-type by cell matrix
# es.max = sctype_score(scRNAseqData = scte[["RNA"]]@scale.data, scaled = TRUE, 
#                       gs = gs_list$gs_positive, gs2 = gs_list$gs_negative) 

# # merge by cluster
# cL_resutls = do.call("rbind", lapply(unique(scte@meta.data$seurat_clusters), function(cl){
#     es.max.cl = sort(rowSums(es.max[ ,rownames(scte@meta.data[scte@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
#     head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(scte@meta.data$seurat_clusters==cl)), 10)
# }))
# sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# # set low-confident (low ScType score) clusters to "unknown"
# sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"


# scte@meta.data$customclassif = ""
# for(j in unique(sctype_scores$cluster)){
#   cl_type = sctype_scores[sctype_scores$cluster==j,]; 
#   scte@meta.data$customclassif[scte@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
# }

# ## save cell type
# cell_types<-scte[['customclassif']]
